In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import os
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_list = []
class_list = []
DATASET = '/content/drive/MyDrive/salim ridho/data set/cell_images/cell_images'

In [ ]:
IMG_SIZE = 128

In [ ]:
CATEGORIES = ["Parasitized","Uninfected"]

In [ ]:
for category in CATEGORIES:  
    path = os.path.join(DATASET,category)  # create path to KATEGORI
    for img in os.listdir(path):  # iterate over each image per KATEGORI
        img_array = cv2.imread(os.path.join(path,img))  # convert to array
        plt.imshow(img_array, cmap='gray')  # graph it
        plt.show()  # display!

        break  # we just want one for now so break
    break  #...and one more!

In [ ]:
training_data = []
def create_training_data():
	for category in CATEGORIES :
		path = os.path.join(DATASET, category)
		class_num = CATEGORIES.index(category)
		for img in os.listdir(path):
			try :
				img_array= cv2.imread(os.path.join(path, img))
				#image filtering
				#kernel = np.ones((2,2),np.float32)/255
				#dst = cv2.filter2D(img_array,-1,kernel)
				# global thresholding
				#ret1,th1 = cv2.threshold(dst,2,255,cv2.THRESH_BINARY)
				#resize
				new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
			  
				training_data.append([new_array, class_num])
			except Exception as e:
				pass

create_training_data()
random.shuffle(training_data)

In [ ]:
x_train = [] #features
y_train = [] #labels

for features, label in training_data:
	x_train.append(features)
	y_train.append(label)

x_train = np.array(x_train).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [ ]:
x_train.shape

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_train, y_train, test_size = 0.2)

In [ ]:
from keras.utils import np_utils
ytrain = np_utils.to_categorical(ytrain, num_classes = 2)
ytest = np_utils.to_categorical(ytest, num_classes = 2)

In [ ]:
ytrain.shape

In [ ]:
ytest.shape

In [ ]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,MaxPooling2D

In [ ]:
NUM_CLASSES = 2

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=[IMG_SIZE,IMG_SIZE] + [3]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='sigmoid'))

model.layers[0].trainable = False

model.layers[0].trainable = False

model.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)

model.summary()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.987 ):
            print("\nReached 98.7% accuracy so cancelling training!")
            self.model.stop_training = True 

In [ ]:
callbacks=myCallback()
history = model.fit(xtrain,ytrain,
        batch_size = 80,
        epochs=50,
        validation_data=(xtest, ytest),
        callbacks = callbacks,
        verbose=1, shuffle=True)

In [ ]:
def plot_accuracy_and_loss(history):
    hist = history.history
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = range(len(acc))
    f, ax = plt.subplots(1,2, figsize=(14,6))
    ax[0].plot(epochs, acc, 'g', label='Training accuracy')
    ax[0].plot(epochs, val_acc, 'r', label='Validation accuracy')
    ax[0].set_title('Training and validation accuracy')
    ax[0].legend()
    ax[1].plot(epochs, loss, 'g', label='Training loss')
    ax[1].plot(epochs, val_loss, 'r', label='Validation loss')
    ax[1].set_title('Training and validation loss')
    ax[1].legend()
    plt.show()
plot_accuracy_and_loss(history)


In [ ]:
train_loss, train_acc = model.evaluate(xtrain,  ytrain, verbose=2)
test_loss, test_acc = model.evaluate(xtest,  ytest, verbose=2)
print("Training accuracy= ", train_acc)
print("Validation accuracy= ", test_acc)

In [ ]:
from sklearn.metrics import confusion_matrix
pred = model.predict(xtest)
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(ytest,axis = 1)

In [ ]:
CM = confusion_matrix(y_true, pred)
from mlxtend.plotting import plot_confusion_matrix

fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_true, pred))

In [ ]:
from sklearn import metrics
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, pred, digits=2,  target_names = ["normal","parasit"]))

In [ ]:
y_true

In [ ]:
model_feat = Model(inputs=model.input,outputs=model.get_layer

('dense_1').output)

feat_train = model_feat.predict(xtrain)
print(feat_train.shape)


feat_test = model_feat.predict(xtest)
print(feat_test.shape)

In [ ]:
# build the lightgbm model
import lightgbm as lgb

lgbc = lgb.LGBMClassifier()
lgbc.fit(feat_train,np.argmax(ytrain,axis=1))

print('fitting done !!!')

In [ ]:
print('LightGBM Model Training accuracy score: {0:0.4f}'.format(lgbc.score(feat_train,np.argmax(ytrain,axis=1))))

In [ ]:
print('LightGBM Model Testing accuracy score: {0:0.4f}'.format(lgbc.score(feat_test,np.argmax(ytest,axis=1))))

In [ ]:
Predlgbm = lgbc.predict(feat_test)
CM = confusion_matrix(y_true, Predlgbm)
from mlxtend.plotting import plot_confusion_matrix

fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
plt.show()

In [ ]:
model.save('my_model.h5') 